<a href="https://colab.research.google.com/github/Mithunprom/-Data-Science_codding_challenge/blob/master/DataScience_coding_prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
#This is a coding challenge problem
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers


embedding_dim = 100
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=160000
test_portion=.1

corpus = []

In [64]:
from pandas import read_csv
import io
Data=read_csv(io.BytesIO(uploaded['Eluvio_DS_Challenge.csv']))
Data['over_18']=(Data['over_18'] == 'TRUE').astype(int)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [66]:
Data.shape

(509236, 8)

In [5]:

!wget --no-check-certificate \
    https://workable.com/nr?l=https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F15X00ZWBjla7qGOIW33j8865QdF89IyAk%2Fview%3Fusp%3Dsharing%255C\
    -O /tmp/training_data.csv





--2019-11-03 00:31:33--  https://workable.com/nr?l=https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F15X00ZWBjla7qGOIW33j8865QdF89IyAk%2Fview%3Fusp%3Dsharing%255C
Resolving workable.com (workable.com)... 104.17.162.47, 104.17.163.47, 2606:4700::6811:a32f, ...
Connecting to workable.com (workable.com)|104.17.162.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/tmp/training_data.csv’

/tmp/training_data.     [ <=>                ]  22.35K  --.-KB/s    in 0s      

2019-11-03 00:31:33 (86.9 MB/s) - ‘/tmp/training_data.csv’ saved [22890]



In [67]:

num_sentences = 0
list_item=Data.iloc[:,4]
this_label=Data.iloc[:,5]
# if this_label=='True':
#     list_item.append(1)
# else:
#     list_item.append(0)
# num_sentences = num_sentences + 1
# corpus.append(list_item)
print(list_item[0])
print(this_label[0])
print(list_item.shape)
print(this_label.shape)
n=len(Data)
n

Scores killed in Pakistan clashes
0
(509236,)
(509236,)


509236

In [0]:
num_sentences =len(Data)
corpus=[]
for i in range(num_sentences):
  corpus.append([Data.iloc[i,4],Data.iloc[i,5]])

In [69]:
print(num_sentences)
print(len(corpus))
print(corpus[0])

509236
509236
['Scores killed in Pakistan clashes', 0]


In [0]:
training_size=num_sentences
sentences=[]
labels=[]
random.shuffle(corpus)
for x in range(training_size):
    sentences.append(corpus[x][0])
    labels.append(corpus[x][1])


tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
vocab_size=len(word_index)

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

split = int(test_portion * training_size)

test_sequences = padded[0:split]
training_sequences = padded[split:training_size]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [71]:
print(vocab_size)
print(word_index['i'])
# Expected Output
# 138858
# 1

108705
195


In [73]:
# Note this is the 100 dimension version of GloVe from Stanford
# I unzipped and hosted it on my site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

--2019-11-03 01:42:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   173MB/s    in 1.9s    

2019-11-03 01:42:22 (173 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [0]:
training_labels=np.array(training_labels).reshape(num_sentences-split ,1)
test_labels=np.array(test_labels).reshape(split ,1)


In [75]:
print(training_sequences.shape)
print(training_labels.shape)
print(num_epochs)
print(test_sequences.shape)
print(test_labels.shape)

(458313, 16)
(458313, 1)
50
(50923, 16)
(50923, 1)


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 50
history = model.fit(training_sequences, training_labels, epochs=num_epochs, validation_data=(test_sequences, test_labels), verbose=2)

print("Training Complete")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 16, 100)           10870600  
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 12, 64)            32064     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 10,935,753
Trainable params: 65,153
Non-trainable params: 10,870,600
_____________________________________

In [0]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


# Expected Output
# A chart where the validation loss does not increase sharply!